# Formation ML Flow

L'objectif de cette formation est de vous apprendre à utiliser ML Flow, un dashboard pour expériences de Machine Learning. Cette formation vous montrera pas à pas comment lancer une expérience de Machine Learning et surveiller les paramètres, métriques et resultats de votre expérience grâce à ML Flow. 

Pour cela, la formation s'appuiera sur le célèbre jeu de donnée Iris, qui classifie 150 echantillons de fleur en 3 espèces d'iris : Iris setosa, Iris versicolor et Iris virginica. Le dataset comprend les caractéristiques de chaque échantillon (longueur et largeur des pétales et longueur et largeur des sépales) et nous allons donc essayer de créer plusieurs modèles qui essaieront de classifier les échantillons d'iris selon leurs caractéristiques. 

Commençons par importer les librairies qui vont nous servir :

In [ ]:
import mlflow
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 

from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

## Ouverture et visualisation du dataset Iris

A présent ouvrons le fichier contenant le dataset et affichons les données concernant les 5 premiers échantillons :

In [ ]:
dataset = pd.read_csv('iris.csv')
dataset.head()

Affichons maintenant quelques informations sur les caractéristiques des fleurs :

In [ ]:
dataset.describe()

Affichons le nombre d'échantillons pour chacune des espèces d'iris :

In [ ]:
dataset.groupby('class').size()

Affichons maintenant le nuage de points de chaque paire de caractéristiques. Sur la diagonale, on retrouve un histogramme du nombre d'échantillons selon la valeur de la caractéristique. On peut remarquer certaines structures, par exemple des groupes, dans les relations entre certaines caractéristiques.

In [ ]:
pd.plotting.scatter_matrix(dataset,figsize=(15,7))
plt.suptitle('Scatter matrix of the Iris dataset')
plt.show()

## ML Flow : premiers pas avec un modèle aléatoire

Nous allons maintenant réaliser un premier exemple de modèle de classification très simple : le modèle aléatoire. Ce modèle attribuera aléatoirement une classe à chaque échantillon du dataset. Nous visualiserons ensuite ses performances (pas très bonnes, on l'imagine) avec ML FLow.

Commençons par séparer le dataset en 2 variables : les 4 caractéristiques d'une part (X), et l'espèce d'iris d'autre part (Y).

In [ ]:
array = dataset.values
X = array[:,0:4]
Y = array[:,4]

Construisons notre modèle aléatoire :

In [ ]:
from sklearn.dummy import DummyClassifier
dummy = DummyClassifier(strategy="uniform")
dummy.fit(X, Y)

Affichons ses prédictions pour les premiers échantillons :

In [ ]:
Y_pred = dummy.predict(X)
for i in range(5):
    print(X[i], ', vraie classe = ', Y[i], ', prédiction : ', Y_pred[i])

Nous allons maintenant enregistrer les performances de ce magnifique modèle avec ML Flow. Pour l'instant executez les cellules suivantes "bêtement", nous expliquerons chaque ligne ensuite.

In [ ]:
with mlflow.start_run(run_name='Modele_Aleatoire'):
    mlflow.log_param('MODEL_NAME', 'DummyClassifier')
    mlflow.log_metric('Accuracy', accuracy_score(Y, Y_pred))
    mlflow.log_artifact('iris.csv')

Maintenant ouvrez un terminal et naviguez jusqu'au dossier ```formation-mlflow```. Affichez le contenu du dossier. Vous devriez y trouver un nouveau dossier nommé ```mlflow```. Ce dossier contient toutes les informations que nous avons enregistré précédemment.

Toujours dans le dossier ```formation-mlflow```, entrez la commande ```mlflow ui```. Le message ```Serving on http://XXXXXXX:5000``` devrait s'afficher. Ouvrez maintenant un navigateur web et ouvrez une fenêtre vers l'url ```http://localhost:5000/```. Vous devriez voir apparaître le dashboard suivant :

![dashboard](MLFlow.png)

La ligne au milieu du tableau, comportant le nom ```Modele_Aleatoire``` correspond aux données que nous avons enregistré précédemment. Vous pouvez remarquer dans la colonne ```Parameters``` le paramètre  ```MODEL_NAME``` que nous avons enregistré, ainsi que la métrique ```Accuracy``` dans la colonne ```Metrics```. Maintenant cliquez sur cette ligne et vous trouverez le détail des informations concernant ce ```run```. Descendez jusqu'à la section ```Artifacts``` où vous pourrez voir une copie du fichier ```iris.csv```. En cliquant dessus vous pourrez le visualiser en entier.

Revenons maintenant sur la façon dont nous avons enregistré ces informations avec ML FLow.

ML Flow permet d'enregistrer puis visualiser 3 types de choses :
* des **paramètres** : ce sont des valeurs (int, float, string, ...) qui ne varient pas au cours d'un "run", ici ```MODEL_NAME``` est un paramètre.
* des **métriques** : ce sont des valeurs qui peuvent varier au cours du run, ici ```Accuracy``` est une métrique.
* des **fichiers** : ces fichiers peuvent prendre n'importe quelle forme (png, jpeg, gif, txt, ...) et ne sont pas modifiables au cours du run. Ici nous avons fait une copie du fichier ```iris.csv```.

Toutes ces variables sont regroupées dans un même 'run', ici 'Modele_Aleatoire'. Les 'runs' peuvent être regroupés en 'experiences', ici nommée ```Default``` dans la colonne de gauche. Nous verons comment changer le nom de l'expérience par la suite.

Reprenons les lignes de code qui nous ont permis de faire tout ça :

In [ ]:
# 1. Création du 'run' ML FLow, nommé ici 'Modele_Aleatoire'
#    Tous les paramètres, métriques ou fichiers que nous enregistrerons ensuite
#    seront enregistré dans ce 'run', qui correspond à une ligne du tableau
with mlflow.start_run(run_name='Modele_Aleatoire'):

    # 2. Enregistrement d'un paramètre
    mlflow.log_param('MODEL_NAME', 'DummyClassifier')

    # 3. Enregistrement d'une métrique
    mlflow.log_metric('Accuracy', accuracy_score(Y, Y_pred))

    # 4. Enregistrement d'un fichier
    mlflow.log_artifact('iris.csv')

Nous allons maintenant entraîner différents modèles sur le jeu de données et voir comment nous pouvons les comparer facilement avec ML FLow.

## Comparaison de plusieurs modèles avec ML Flow

ML Flow permet d'enregistrer puis visualiser 3 types de choses :
* des **paramètres** : ce sont des valeurs (int, float, string, ...) qui ne varient pas au cours d'un "run".
* des **métriques** : ce sont des valeurs qui peuvent varier au cours du run.
* des **fichiers** : ces fichiers peuvent prendre n'importe quelle forme (png, jpeg, gif, txt, ...) et ne sont pas modifiables au cours du run.

log training time

Now it is time to create some models of the data and estimate their accuracy on unseen data.

Here is what we are going to cover in this step:

Separate out a validation dataset.
Set-up the test harness to use 10-fold cross validation.
Build multiple different models to predict species from flower measurements
Select the best model.

In [ ]:
sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

In [ ]:
from sklearn.model_selection import train_test_split

array = dataset.values
X = array[:,0:4]
y = array[:,4]
X_train, X_val, Y_train, Y_val = train_test_split(X, y, test_size=0.20, random_state=1)

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
model = LogisticRegression(solver='liblinear', multi_class='ovr')
kfold = sklearn.model_selection.StratifiedKFold(n_splits=10, random_state=1, shuffle=True)
cv_results = sklearn.model_selection.cross_val_score(model, X_train, Y_train, cv=kfold, scoring='accuracy')
print('%s: %f (%f)' % ('LogisticRegression', cv_results.mean(), cv_results.std()))

In [ ]:
model.fit(X_train, Y_train)
predictions = model.predict(X_validation)

In [ ]:
predictions

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_scoreb
# Evaluate predictions
print(accuracy_score(Y_validation, predictions))
print(confusion_matrix(Y_validation, predictions))
print(classification_report(Y_validation, predictions))
